### Project of the Mining of Massive Datasets Challenge
# Torch and TensorFlow
### Michał Kempka, Artur Lasowski, Marek Wydmuch
#### 2.12.2016

In [ ]:
require 'image';
torchLogo = image.load('torch.jpg')
itorch.image({torchLogo})

## Lua in 5 minutes or less

In [ ]:
x = 5           -- All numbers are doubles.
xStr = 'five'
xBool = true

local y = 5     -- Variables are global by default. 

y = nil         -- Undefines t, Lua has garbage collection.

function add(a, b)
    return a + b
end

In [ ]:
-- Tables are hash-lookup dictionaries. 
-- Serve as tables, list, maps, objects and pacakges.
-- Indices start at 1.

t = { 5, 10, 15, a = 25, b = 30, [xStr] = 5, [x] = xStr, 20 }
t.d = 35
t['ten'] = 10
t[10] = 'ten'

function t:add() -- function t.add(self)
    return self.a + self.b
end

t.ab = t:add()   -- t.add(t)
  
print(t)

In [ ]:
-- Only nil and false are falsy. 0 and '' are true!
x = 0
if x then 
    print('1: ' .. x .. ' is true') 
end

if x ~= 0 then 
    print('2: ' .. x .. ' is true') 
end

-- Google: Lua in 15 minutes

## Torch Tensors

In [ ]:
require 'torch';

-- Constructors
x = torch.Tensor(3, 5)
x:fill(2)
x[1][1] = 4
x[3][3] = 4

y = torch.eye(3, 5) -- rand, ones, zeros, ...

xCopy = x:clone() -- z = torch.Tensor(x:size()):copy(x)

print('x:')
print(x)

-- BLAS and element wise
z = x * y:t()
z:mm(x, y:t()) -- add, dot, ...
z:pow(2) -- abs, round, ...

print('z: ')
print(z)

In [ ]:
require 'cutorch';

x = x:cuda()
y = y:cuda()
xCuda = torch.CudaTensor(3, 5)

z:mm(x, y:t()) -- computed on GPU

## Logistic regression

In [ ]:
require 'nn';
require 'optim';
require 'math';

-- Load data
dofile 'util.lua';

datasetSeparable = readFile("dataset1")
datasetInseparable = readFile("dataset2")

print(datasetSeparable)

In [ ]:
function splitDataset(dataset, p)
    p = p or 0.8
    
    trainSize = torch.round(dataset.size * p)
    testSize = dataset.size - trainSize

    train = {
        size = trainSize,
        data = dataset.data[{{1,trainSize}}]:double(),
        label = dataset.label[{{1,trainSize}}]
    }

    test = {
        size = testSize,
        data = dataset.data[{{trainSize + 1, trainSize + testSize}}]:double(),
        label = dataset.label[{{trainSize + 1, trainSize + testSize}}]
    }

    return {train=train, test=test}
    
end

_datasetSeparable = splitDataset(datasetSeparable)
_datasetInseparable = splitDataset(datasetInseparable)
print(_datasetSeparable)

In [ ]:
-- Logistic regresion

model = nn.Sequential()
model:add(nn.Linear(2,1))
model:add(nn.Sigmoid())
criterion = nn.BCECriterion()

x, dldx = model:getParameters()

In [ ]:
-- Some parameters

batchSize = 60
epochs = 25

config = {
   learningRate = 0.1
}

In [ ]:
-- Epoch

epoch = function(dataset)
    local currentLoss = 0
    local count = 0
    local shuffle = torch.randperm(dataset.size)
    
    for d = 1, dataset.size, batchSize do
        -- setup inputs and targets for this mini-batch
        local size = math.min(d + batchSize - 1, dataset.size) - d
        local inputs = torch.Tensor(size, 2)
        local targets = torch.Tensor(size)
        
        for i = 1, size do
            local input = dataset.data[shuffle[i + d]]
            local target = dataset.label[shuffle[i + d]]
            
            inputs[i] = input
            targets[i] = target
        end
        
        local feval = function(xNew)
            -- reset data
            if x ~= xNew then x:copy(xNew) end
            dldx:zero()

            local loss = criterion:forward(model:forward(inputs), targets)
            model:backward(inputs, criterion:backward(model.output, targets))

            return loss, dldx
        end
        
        _, fs = optim.sgd(feval, x, config)
        -- fs is a table containing value of the loss function
        -- (just 1 value for the SGD optimization)
        count = count + 1
        currentLoss = currentLoss + fs[1] -- absence of ++/-- and +=/-= operators
    end

    -- normalize loss
    return currentLoss / count
end

In [ ]:
-- Evaluation
eval = function(dataset)
    local count = 0
    
    for i = 1, dataset.size, batchSize do
        local size = math.min(i + batchSize - 1, dataset.size) - i
        local inputs = dataset.data[{{i, i + size - 1}}]
        local targets = dataset.label[{{i, i + size - 1}}]
        
        local outputs = model:forward(inputs)
        outputs:round()
        local correct = outputs:eq(targets):sum()
        count = count + correct
    end

    return count / dataset.size
end

In [ ]:
-- Train
function train(dataset, epochs)
    epochs = epochs or 1
    for i = 1, epochs do
        print(string.format('Epoch: %d loss: %3f', i, epoch(dataset.train)))
        print(string.format('Train accuracy: %3f', eval(dataset.train)))
        print(string.format('Validation accuracy: %3f', eval(dataset.test)))
    end
end

In [ ]:
-- FUN
torch.manualSeed(9876)
model:reset()

train(_datasetInseparable, 20)

In [ ]:
-- Saving & Loading models
paths = require 'paths'
filename = paths.concat(paths.cwd(), 'model.net')

torch.save(filename, model)
modelLoaded = torch.load(filename)


## Neural networks

In [ ]:
-- Simple 3-layer neural network

model = nn.Sequential()
model:add(nn.Linear(2,10))
model:add(nn.ReLU()) -- nn.ReLU
model:add(nn.Linear(10,10))
model:add(nn.ReLU())
model:add(nn.Linear(10,1))
model:add(nn.Sigmoid())
criterion = nn.BCECriterion()

x, dldx = model:getParameters()

In [ ]:
-- FUN
torch.manualSeed(9876)
model:reset()

train(_datasetInseparable, 20)

## Multiclass convolution neural network

In [ ]:
mnist = require 'mnist'
mnistDataset = mnist.traindataset()

itorch.image(mnistDataset.data[{{1, 12}}])
print(mnistDataset.label[{{1, 12}}])

_mnistDataset = splitDataset(mnistDataset)

In [ ]:
model = nn.Sequential()
model:add(nn.Reshape(28 * 28))
model:add(nn.Linear(28 * 28, 30))
model:add(nn.Tanh())
model:add(nn.Linear(30, 10))
model:add(nn.LogSoftMax())
criterion = nn.ClassNLLCriterion()


In [ ]:
-- input dimensions
nfeats = 3
width = 32
height = 32
ninputs = nfeats*width*height

-- number of hidden units (for MLP only):
nhiddens = ninputs / 2

-- hidden units, filter sizes (for ConvNet only):
nstates = {64,64,128}
filtsize = 5
poolsize = 2

-- Typical modern convolution network (conv + relu + pool)

model = nn.Sequential()

model:add(nn.SpatialConvolutionMM(nfeats, nstates[1], filtsize, filtsize))
model:add(nn.ReLU())
model:add(nn.SpatialMaxPooling(poolsize,poolsize,poolsize,poolsize))

model:add(nn.SpatialConvolutionMM(nstates[1], nstates[2], filtsize, filtsize))
model:add(nn.ReLU())
model:add(nn.SpatialMaxPooling(poolsize,poolsize,poolsize,poolsize))

model:add(nn.View(nstates[2]*filtsize*filtsize))
model:add(nn.Dropout(0.5))
model:add(nn.Linear(nstates[2]*filtsize*filtsize, nstates[3]))
model:add(nn.ReLU())
model:add(nn.Linear(nstates[3], noutputs))

model:add(nn.Linear(30, 10))
model:add(nn.LogSoftMax())

criterion = nn.ClassNLLCriterion()

In [ ]:
x, dldx = model:getParameters()

config = {
   learningRate = 1e-2,
   learningRateDecay = 1e-4,
   weightDecay = 1e-3,
   momentum = 1e-4
}

In [ ]:
-- Epoch
epoch = function(dataset)
    local currentLoss = 0
    local count = 0
    local shuffle = torch.randperm(dataset.size)
    
    for d = 1, dataset.size, batchSize do
        -- setup inputs and targets for this mini-batch
        local size = math.min(d + batchSize - 1, dataset.size) - d
        local inputs = torch.Tensor(size, 28, 28) -- HERE!
        local targets = torch.Tensor(size)
        
        for i = 1, size do
            local input = dataset.data[shuffle[i + d]]
            local target = dataset.label[shuffle[i + d]]
            
            inputs[i] = input
            targets[i] = target
        end
        targets:add(1)
        
        local feval = function(xNew)
            if x ~= xNew then x:copy(xNew) end
            dldx:zero()

            local loss = criterion:forward(model:forward(inputs), targets)
            model:backward(inputs, criterion:backward(model.output, targets))

            return loss, dldx
        end
        
        _, fs = optim.sgd(feval, x, config)
        count = count + 1
        currentLoss = currentLoss + fs[1]
    end

    -- normalize loss
    return currentLoss / count
end

In [ ]:
eval = function(dataset)
    local count = 0
    
    for i = 1, dataset.size, batchSize do
        local size = math.min(i + batchSize - 1, dataset.size) - i
        local inputs = dataset.data[{{i, i + size - 1}}]
        local targets = dataset.label[{{i, i + size - 1}}]:long()
        
        local outputs = model:forward(inputs)
        local _, outputs = torch.max(outputs, 2)
        outputs:add(-1)
        
        local correct = outputs:eq(targets):sum()
        count = count + correct
    end

    return count / dataset.size
end

In [ ]:
-- FUN
torch.manualSeed(9876)
model:reset()

train(_mnistDataset, 30)